In [1]:
import pandas as pd
import os
import requests
import time

/Users/jackyhe/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# Replace these with your actual handle and app password
BLUESKY_HANDLE = "jackyhe.bsky.social"
BLUESKY_APP_PASSWORD = "#"

# Create a session
response = requests.post(
    "https://bsky.social/xrpc/com.atproto.server.createSession",
    json={"identifier": BLUESKY_HANDLE, "password": BLUESKY_APP_PASSWORD},
)
response.raise_for_status()
session = response.json()
access_token = session["accessJwt"]

headers = {
    "Authorization": f"Bearer {access_token}"
}

# Example: Fetch posts
response = requests.get(
    "https://bsky.social/xrpc/app.bsky.feed.getTimeline",
    headers=headers
)

refresh_token = session["refreshJwt"]

# Refresh the session
refresh_response = requests.post(
    "https://bsky.social/xrpc/com.atproto.server.refreshSession",
    headers={"Authorization": f"Bearer {refresh_token}"}
)
refresh_response.raise_for_status()
new_session = refresh_response.json()
new_access_token = new_session["accessJwt"]

In [3]:
# HurtLex is a lexicon of offensive, aggressive, and hateful words. https://github.com/valeriobasile/hurtlex
# Load HurtLex English lexicon
hurtlex_path = os.path.join('hurtlex_EN.tsv')
hurtlex_df = pd.read_csv(hurtlex_path, sep='\t')

# Filter for 'conservative' level if desired
hurtlex_keywords = set(hurtlex_df[hurtlex_df['level'] == 'conservative']['lemma'].str.lower())

In [4]:
# Function to label posts
def label_post(text):
    if pd.isnull(text):
        return 'Not Hate Speech'
    text_lower = text.lower()
    for keyword in hurtlex_keywords:
        if keyword in text_lower:
            return 'Hate Speech'
    return 'Not Hate Speech'

In [5]:
def fetch_posts(query, limit=100, cursor=None):
    url = "https://api.bsky.app/xrpc/app.bsky.feed.searchPosts"
    params = {
        "q": query,
        "limit": limit
    }
    if cursor:
        params["cursor"] = cursor
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching posts: {e}")
        return None

In [6]:

def is_race_related(text):
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in ['race', 'ethnicity', 'ethnic', 'racial', 'racism', 'racist', 'black', 'white', 'asian', 'latino', 'hispanic', 'african', 'european', 'indian', 'native', 'minority', 'majority', 'diversity', 'multicultural', 'heritage', 'ancestry'])


def collect_and_label_posts(target_count_per_label=50):
    queries = ["the", "and", "to", "of", "a"]  # Common words to fetch diverse posts
    collected_posts = []
    label_counts = {"Hate Speech": 0, "Not Hate Speech": 0}

    for query in queries:
        cursor = None
        while True:
            data = fetch_posts(query, limit=100, cursor=cursor)
            if not data or "posts" not in data:
                break

            for post in data["posts"]:
                text = post.get("record", {}).get("text", "")
                uri = post.get("uri", "")
                if not text or not uri:
                    continue

                if not is_race_related(text): continue
                label = label_post(text)
                if label and label_counts[label] < target_count_per_label:
                    post_url = f"https://bsky.app/profile/{uri.split('/')[2]}/post/{uri.split('/')[-1]}"
                    collected_posts.append({"post_url": post_url, "label": label})
                    label_counts[label] += 1

                    # Print progress
                    print(f"Collected {len(collected_posts)} posts: "
                          f"Hate Speech: {label_counts['Hate Speech']}, "
                          f"Not Hate Speech: {label_counts['Not Hate Speech']}")

                if all(count >= target_count_per_label for count in label_counts.values()):
                    break

            if all(count >= target_count_per_label for count in label_counts.values()):
                break

            cursor = data.get("cursor")
            if not cursor:
                break
            time.sleep(1)  # Respectful delay to avoid rate limiting

    print(f"Final counts - Total posts: {len(collected_posts)}, "
          f"Hate Speech: {label_counts['Hate Speech']}, "
          f"Not Hate Speech: {label_counts['Not Hate Speech']}")
    return collected_posts

In [7]:
# Execute the data collection
dataset = collect_and_label_posts()

Collected 1 posts: Hate Speech: 1, Not Hate Speech: 0
Collected 2 posts: Hate Speech: 2, Not Hate Speech: 0
Collected 3 posts: Hate Speech: 2, Not Hate Speech: 1
Collected 4 posts: Hate Speech: 3, Not Hate Speech: 1
Collected 5 posts: Hate Speech: 3, Not Hate Speech: 2
Collected 6 posts: Hate Speech: 4, Not Hate Speech: 2
Collected 7 posts: Hate Speech: 5, Not Hate Speech: 2
Collected 8 posts: Hate Speech: 5, Not Hate Speech: 3
Collected 9 posts: Hate Speech: 6, Not Hate Speech: 3
Collected 10 posts: Hate Speech: 6, Not Hate Speech: 4
Collected 11 posts: Hate Speech: 7, Not Hate Speech: 4
Collected 12 posts: Hate Speech: 8, Not Hate Speech: 4
Collected 13 posts: Hate Speech: 9, Not Hate Speech: 4
Collected 14 posts: Hate Speech: 9, Not Hate Speech: 5
Collected 15 posts: Hate Speech: 10, Not Hate Speech: 5
Collected 16 posts: Hate Speech: 11, Not Hate Speech: 5
Collected 17 posts: Hate Speech: 12, Not Hate Speech: 5
Collected 18 posts: Hate Speech: 13, Not Hate Speech: 5
Collected 19 po

In [8]:
# Save to CSV
df = pd.DataFrame(dataset)
df.to_csv("test-data/bluesky_race_hate_speech_dataset.csv", index=False)
print("Dataset saved to 'bluesky_race_hate_speech_dataset.csv'")

Dataset saved to 'bluesky_race_hate_speech_dataset.csv'
